In [1]:
import pandas as pd
import numpy as np

In [2]:
c18=pd.read_excel('datasets/C-18.xlsx',skiprows=6,header=None,engine='openpyxl')

In [3]:
c13=pd.read_excel('datasets/C-13.xls',skiprows=7,header=None)

## obtain useful data from C-13 and C-18 for age-groups

- formulas used:

In [4]:
# STATE_NAMES=[list(np.unique(c18.iloc[:,2].values))]
STATE_NAMES=[]
for state in c18.iloc[:,2].values:
    if not (state in STATE_NAMES):
        STATE_NAMES.append(state)
AGE_GROUPS=list(c18.iloc[1:10,4].values)
# although it is a bit of manual work but it is worth the efforts
AGE_GROUP_RANGES=[list(range(5,10)),list(range(10,15)),list(range(15,20)),list(range(20,25)),list(range(25,30)),list(range(30,50)),list(range(50,70)),list(range(70,100))+['100+'],['Age not stated']]

In [5]:
useful_data=[]
for i,state in enumerate(STATE_NAMES):
    for j,age_grp in enumerate(AGE_GROUPS):
        # this list is to get only the years in the particular age-group

        true_false_list=[]
        for single_year_age in c13.iloc[:,4].values:
            if single_year_age in AGE_GROUP_RANGES[j]:
                true_false_list.append(True)
            else:
                true_false_list.append(False)

        # here i is the state code
        male_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,6].values.sum()
        female_pop=c13[(c13.loc[:,1]==i) & (true_false_list)].iloc[:,7].values.sum()
        
        # tri
        tri_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,9]
        tri_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,10]

        #bi
        bi_male=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,6] - tri_male
        bi_female=c18[(c18.iloc[:,0]==i) & (c18.iloc[:,4]==age_grp) & (c18.iloc[:,3]=='Total')].iloc[0,7] - tri_female

        #uni
        uni_male=male_pop-bi_male-tri_male
        uni_female=female_pop-bi_female-tri_female

        item={
            'state-code':i,
            'state-name':state,
            'age-group':age_grp,
            'age-group-male-pop':male_pop,
            'age-group-female-pop':female_pop,
            'tri-male-ratio':tri_male/male_pop,
            'tri-female-ratio':tri_female/female_pop,
            'bi-male-ratio':bi_male/male_pop,
            'bi-female-ratio':bi_female/female_pop,
            'uni-male-ratio':uni_male/male_pop,
            'uni-female-ratio':uni_female/female_pop
        }

        useful_data.append(item)

In [6]:
df=pd.DataFrame(useful_data)

## age-analysis 

In [7]:
tri_list=[]
bi_list=[]
uni_list=[]
for i in range(36):
    male_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,5]].values
    female_values=df[df['state-code']==i].sort_values(by='tri-male-ratio',ascending=False).iloc[0,[2,6]].values
    tri_item={
        'state/ut':STATE_NAMES[i],
        'age-group-male':male_values[0],
        'ratio-of-3-male':male_values[1],
        'age-group-female':female_values[0],
        'ratio-of-3-female':female_values[1]
    }

    tri_list.append(tri_item)

    male_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,7]].values
    female_values=df[df['state-code']==i].sort_values(by='bi-male-ratio',ascending=False).iloc[0,[2,8]].values
    bi_item={
        'state/ut':STATE_NAMES[i],
        'age-group-male':male_values[0],
        'ratio-of-2-male':male_values[1],
        'age-group-female':female_values[0],
        'ratio-of-2-female':female_values[1]
    }

    bi_list.append(bi_item)

    male_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,9]].values
    female_values=df[df['state-code']==i].sort_values(by='uni-male-ratio',ascending=False).iloc[0,[2,10]].values
    uni_item={
        'state/ut':STATE_NAMES[i],
        'age-group-male':male_values[0],
        'ratio-of-1-male':male_values[1],
        'age-group-female':female_values[0],
        'ratio-of-1-female':female_values[1]
    }

    uni_list.append(uni_item)

In [8]:
tri_df=pd.DataFrame(tri_list)
bi_df=pd.DataFrame(bi_list)
uni_df=pd.DataFrame(uni_list)

In [9]:
tri_df.to_csv('outputs/age-gender-3.csv',index=False)
bi_df.to_csv('outputs/age-gender-2.csv',index=False)
uni_df.to_csv('outputs/age-gender-1.csv',index=False)

## observations

- in almost all cases both highest ratio female and male age-groups are same.
- interestingly in only one language case for all states '5-9' age group dominates, and it is also quite intutive; since at that early stage in life children only speak their mother toung only 